In [16]:
import os
import time
import warnings
import pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.random_projection import johnson_lindenstrauss_min_dim
from sklearn.random_projection import SparseRandomProjection
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.exceptions import UndefinedMetricWarning
from itertools import product

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

###############################################################################
# read data from file

def extract_zip(zip_file, extract_to):
    """Extracts a zip file to the specified directory."""
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

def getDataPoints(path):
    """Collects content of all .py files within the given directory."""
    dataPointsList = []
    if not os.path.exists(path):
        print(f"Directory does not exist: {path}")
        return dataPointsList

    for root, dirs, files in os.walk(path):
        for dataPointName in files:
            if dataPointName.endswith(".py"):  # Only consider Python files
                file_path = os.path.join(root, dataPointName)
                with open(file_path, encoding="utf-8") as fileIn:
                    dp = fileIn.read().strip()
                    if dp:  # Ensure the document is not empty
                        dataPointsList.append(dp)
                    else:
                        print(f"Empty file: {file_path}")
    
    if len(dataPointsList) == 0:
        print(f"No valid documents found in directory: {path}")
    
    return dataPointsList

def computeResults(testLabels, predictLabels):
    try:
        precision = precision_score(testLabels, predictLabels)
        recall = recall_score(testLabels, predictLabels)
        accuracy = accuracy_score(testLabels, predictLabels)
        f1 = f1_score(testLabels, predictLabels)
        
        print(f"Precision: {precision}, Recall: {recall}, Accuracy: {accuracy}, F1 Score: {f1}")
    except Exception as e:
        print(f"Error computing metrics: {e}")
        precision = recall = accuracy = f1 = "-"
    return precision, recall, accuracy, f1

###############################################################################
# FLAST

def vectorization(dataPoints, dim=0, eps=0.3):
    countVec = CountVectorizer(stop_words=None)  # Disable stop words filtering
    Z_full = countVec.fit_transform(dataPoints)
    if eps == 0:
        Z = Z_full
    else:
        if dim <= 0:
            dim = johnson_lindenstrauss_min_dim(Z_full.shape[0], eps=eps)
        srp = SparseRandomProjection(n_components=dim)
        Z = srp.fit_transform(Z_full)
    return Z

def classificationDecisionTree(trainData, trainLabels, testData, params):
    # training
    t0 = time.perf_counter()
    clf = DecisionTreeClassifier(
        criterion=params.get("criterion", "gini"),
        splitter=params.get("splitter", "best"),
        max_depth=params.get("max_depth"),
        min_samples_split=params.get("min_samples_split", 2),
        min_samples_leaf=params.get("min_samples_leaf", 1),
    )
    clf.fit(trainData, trainLabels)
    t1 = time.perf_counter()
    trainTime = t1 - t0

    t0 = time.perf_counter()
    predictLabels = clf.predict(testData)
    t1 = time.perf_counter()
    testTime = t1 - t0

    return trainTime, testTime, predictLabels

def decisionTree(outDir, projectBasePath, projectName, kf, dim, eps, params):
    v0 = time.perf_counter()
    
    flakyDir = os.path.join(extractDir, 'flaky')
    nonFlakyDir = os.path.join(extractDir, 'nonFlaky')
    dataPointsFlaky = getDataPoints(flakyDir)
    dataPointsNonFlaky = getDataPoints(nonFlakyDir)
    dataPoints = dataPointsFlaky + dataPointsNonFlaky
    print("Data points before vectorization:",len(dataPoints))
    Z = vectorization(dataPoints, dim=dim, eps=eps)
    dataPointsList = np.array([Z[i].toarray() for i in range(Z.shape[0])])
    dataLabelsList = np.array([1]*len(dataPointsFlaky) + [0]*len(dataPointsNonFlaky))
    v1 = time.perf_counter()
    vecTime = v1 - v0

    # storage
    decisionTree = (dataPointsList, dataLabelsList)
    pickleDumpDecisionTree = os.path.join(outDir, "decision-tree.pickle")
    with open(pickleDumpDecisionTree, "wb") as pickleFile:
        pickle.dump(decisionTree, pickleFile)
    storage = os.path.getsize(pickleDumpDecisionTree)
    os.remove(pickleDumpDecisionTree)

    avgP, avgR, avgAccuracy, avgF1 = 0, 0, 0, 0  # Add avgF1
    avgTPrep, avgTPred = 0, 0
    avgFlakyTrain, avgNonFlakyTrain, avgFlakyTest, avgNonFlakyTest = 0, 0, 0, 0
    successFold, precisionFold = 0, 0
    for (trnIdx, tstIdx) in kf.split(dataPointsList, dataLabelsList):
        trainData, testData = dataPointsList[trnIdx], dataPointsList[tstIdx]
        trainLabels, testLabels = dataLabelsList[trnIdx], dataLabelsList[tstIdx]
        if sum(trainLabels) == 0 or sum(testLabels) == 0:
            print("Skipping fold...")
            print(" Flaky Train Tests", sum(trainLabels))
            print(" Flaky Test Tests", sum(testLabels))
            continue

        successFold += 1
        avgFlakyTrain += sum(trainLabels)
        avgNonFlakyTrain += len(trainLabels) - sum(trainLabels)
        avgFlakyTest += sum(testLabels)
        avgNonFlakyTest += len(testLabels) - sum(testLabels)

        # prepare the data in the right format for Decision Tree
        nSamplesTrainData, nxTrain, nyTrain = trainData.shape
        trainData = trainData.reshape((nSamplesTrainData, nxTrain * nyTrain))
        nSamplesTestData, nxTest, nyTest = testData.shape
        testData = testData.reshape((nSamplesTestData, nxTest * nyTest))

        trainTime, testTime, predictLabels = classificationDecisionTree(trainData, trainLabels, testData, params)
        preparationTime = (vecTime * len(trainData) / len(dataPoints)) + trainTime
        predictionTime = (vecTime / len(dataPoints)) + (testTime / len(testData))
        precision, recall, accuracy, f1 = computeResults(testLabels, predictLabels)

        print(precision, recall, accuracy, f1)  # Add f1 to print statement
        if precision != "-":
            precisionFold += 1
            avgP += precision
            avgF1 += f1  # Aggregate F1 score
        avgR += recall
        avgAccuracy += accuracy
        avgTPrep += preparationTime
        avgTPred += predictionTime

    if precisionFold == 0:
        avgP = avgF1 = "-"  # Adjust for F1
    else:
        avgP /= precisionFold
        avgF1 /= precisionFold  # Average F1 score
    avgR /= successFold
    avgAccuracy /= successFold
    avgTPrep /= successFold
    avgTPred /= successFold
    avgFlakyTrain /= successFold
    avgNonFlakyTrain /= successFold
    avgFlakyTest /= successFold
    avgNonFlakyTest /= successFold

    return (avgFlakyTrain, avgNonFlakyTrain, avgFlakyTest, avgNonFlakyTest, avgP, avgR, avgAccuracy, avgF1, storage, avgTPrep, avgTPred)

if __name__ == "__main__":
    # Parameters setup
    flakyZip = "cleaned_flaky_files.zip"
    nonFlakyZip = "reduced_nonflaky_files.zip"
    extractDir = "extracted"
    outDir = "results-DecisionTree/"
    os.makedirs(outDir, exist_ok=True)
    os.makedirs(extractDir, exist_ok=True)

    numSplit = 30
    testSetSize = 0.2
    kf = StratifiedShuffleSplit(n_splits=numSplit, test_size=testSetSize)

    outFile = "params-DecisionTree.csv"
    with open(os.path.join(outDir, outFile), "w") as fo:
        # Updated the header to match the number of columns
        fo.write("criterion,splitter,max_depth,min_samples_split,min_samples_leaf,avgFlakyTrain,avgNonFlakyTrain,avgFlakyTest,avgNonFlakyTest,precision,recall,accuracy,f1,storage,preparationTime,predictionTime\n")

    # Define the hyperparameter grid
    param_grid = {
        "criterion": ["gini", "entropy"],
        "splitter": ["best", "random"],
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4]
    }

    # Flatten the parameter grid into all possible combinations
    keys, values = zip(*param_grid.items())
    combinations = [dict(zip(keys, v)) for v in product(*values)]

    best_params = None
    best_accuracy = 0

    for params in combinations:
        try:
            results = decisionTree(outDir, extractDir, "projectName", kf, dim=0, eps=0.3, params=params)
            
            # Flatten the results tuple and write to CSV
            results_flat = ','.join(map(str, results))
            print(f"Results: {params['criterion']},{params['splitter']},{params['max_depth']},{params['min_samples_split']},{params['min_samples_leaf']},{results_flat}")
            
            with open(os.path.join(outDir, outFile), "a") as fo:
                fo.write(f"{params['criterion']},{params['splitter']},{params['max_depth']},{params['min_samples_split']},{params['min_samples_leaf']},{results_flat}\n")
            
            # Update the best parameters based on accuracy
            avg_accuracy = results[6]  # Index of accuracy in the results tuple
            if avg_accuracy > best_accuracy:
                best_accuracy = avg_accuracy
                best_params = params

        except Exception as e:
            print(f"Failed for params {params}: {e}")

    print(f"Best hyperparameters found: {best_params} with accuracy: {best_accuracy}")
    print("Decision Tree analysis completed. Results saved to:", outFile)


Data points before vectorization: 94
Precision: 0.6363636363636364, Recall: 0.7777777777777778, Accuracy: 0.6842105263157895, F1 Score: 0.7
0.6363636363636364 0.7777777777777778 0.6842105263157895 0.7
Precision: 0.625, Recall: 0.5, Accuracy: 0.5789473684210527, F1 Score: 0.5555555555555556
0.625 0.5 0.5789473684210527 0.5555555555555556
Precision: 0.7, Recall: 0.7777777777777778, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
0.7 0.7777777777777778 0.7368421052631579 0.7368421052631579
Precision: 0.7142857142857143, Recall: 0.5, Accuracy: 0.631578947368421, F1 Score: 0.5882352941176471
0.7142857142857143 0.5 0.631578947368421 0.5882352941176471
Precision: 0.6666666666666666, Recall: 0.8, Accuracy: 0.6842105263157895, F1 Score: 0.7272727272727273
0.6666666666666666 0.8 0.6842105263157895 0.7272727272727273
Precision: 0.46153846153846156, Recall: 0.6666666666666666, Accuracy: 0.47368421052631576, F1 Score: 0.5454545454545454
0.46153846153846156 0.6666666666666666 0.4736842105

Data points before vectorization: 94
Precision: 0.7272727272727273, Recall: 0.8888888888888888, Accuracy: 0.7894736842105263, F1 Score: 0.8
0.7272727272727273 0.8888888888888888 0.7894736842105263 0.8
Precision: 0.5, Recall: 0.4, Accuracy: 0.47368421052631576, F1 Score: 0.4444444444444444
0.5 0.4 0.47368421052631576 0.4444444444444444
Precision: 0.75, Recall: 0.9, Accuracy: 0.7894736842105263, F1 Score: 0.8181818181818182
0.75 0.9 0.7894736842105263 0.8181818181818182
Precision: 0.8333333333333334, Recall: 0.5, Accuracy: 0.6842105263157895, F1 Score: 0.625
0.8333333333333334 0.5 0.6842105263157895 0.625
Precision: 0.8571428571428571, Recall: 0.6666666666666666, Accuracy: 0.7894736842105263, F1 Score: 0.75
0.8571428571428571 0.6666666666666666 0.7894736842105263 0.75
Precision: 0.75, Recall: 0.6666666666666666, Accuracy: 0.7368421052631579, F1 Score: 0.7058823529411765
0.75 0.6666666666666666 0.7368421052631579 0.7058823529411765
Precision: 0.5625, Recall: 1.0, Accuracy: 0.6315789473684

Precision: 0.6153846153846154, Recall: 0.8, Accuracy: 0.631578947368421, F1 Score: 0.6956521739130435
0.6153846153846154 0.8 0.631578947368421 0.6956521739130435
Precision: 0.875, Recall: 0.7777777777777778, Accuracy: 0.8421052631578947, F1 Score: 0.8235294117647058
0.875 0.7777777777777778 0.8421052631578947 0.8235294117647058
Precision: 0.7142857142857143, Recall: 0.5555555555555556, Accuracy: 0.6842105263157895, F1 Score: 0.625
0.7142857142857143 0.5555555555555556 0.6842105263157895 0.625
Precision: 0.625, Recall: 0.5555555555555556, Accuracy: 0.631578947368421, F1 Score: 0.5882352941176471
0.625 0.5555555555555556 0.631578947368421 0.5882352941176471
Precision: 0.5714285714285714, Recall: 0.4444444444444444, Accuracy: 0.5789473684210527, F1 Score: 0.5
0.5714285714285714 0.4444444444444444 0.5789473684210527 0.5
Precision: 0.5, Recall: 0.3333333333333333, Accuracy: 0.5263157894736842, F1 Score: 0.4
0.5 0.3333333333333333 0.5263157894736842 0.4
Precision: 0.6, Recall: 0.6, Accuracy:

Precision: 0.5833333333333334, Recall: 0.7777777777777778, Accuracy: 0.631578947368421, F1 Score: 0.6666666666666666
0.5833333333333334 0.7777777777777778 0.631578947368421 0.6666666666666666
Precision: 1.0, Recall: 0.6, Accuracy: 0.7894736842105263, F1 Score: 0.75
1.0 0.6 0.7894736842105263 0.75
Precision: 0.8333333333333334, Recall: 0.5, Accuracy: 0.6842105263157895, F1 Score: 0.625
0.8333333333333334 0.5 0.6842105263157895 0.625
Precision: 0.6666666666666666, Recall: 0.6666666666666666, Accuracy: 0.6842105263157895, F1 Score: 0.6666666666666666
0.6666666666666666 0.6666666666666666 0.6842105263157895 0.6666666666666666
Precision: 0.5, Recall: 0.5555555555555556, Accuracy: 0.5263157894736842, F1 Score: 0.5263157894736842
0.5 0.5555555555555556 0.5263157894736842 0.5263157894736842
Precision: 0.75, Recall: 0.6666666666666666, Accuracy: 0.7368421052631579, F1 Score: 0.7058823529411765
0.75 0.6666666666666666 0.7368421052631579 0.7058823529411765
Precision: 0.6363636363636364, Recall: 0

Precision: 0.8888888888888888, Recall: 0.8, Accuracy: 0.8421052631578947, F1 Score: 0.8421052631578947
0.8888888888888888 0.8 0.8421052631578947 0.8421052631578947
Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
0.7777777777777778 0.7 0.7368421052631579 0.7368421052631579
Precision: 0.7142857142857143, Recall: 0.5, Accuracy: 0.631578947368421, F1 Score: 0.5882352941176471
0.7142857142857143 0.5 0.631578947368421 0.5882352941176471
Precision: 0.625, Recall: 0.5555555555555556, Accuracy: 0.631578947368421, F1 Score: 0.5882352941176471
0.625 0.5555555555555556 0.631578947368421 0.5882352941176471
Precision: 0.8888888888888888, Recall: 0.8, Accuracy: 0.8421052631578947, F1 Score: 0.8421052631578947
0.8888888888888888 0.8 0.8421052631578947 0.8421052631578947
Precision: 0.5, Recall: 0.6666666666666666, Accuracy: 0.5263157894736842, F1 Score: 0.5714285714285714
0.5 0.6666666666666666 0.5263157894736842 0.5714285714285714
Precision: 0.6, 

...

Precision: 0.5, Recall: 0.4444444444444444, Accuracy: 0.5263157894736842, F1 Score: 0.47058823529411764
0.5 0.4444444444444444 0.5263157894736842 0.47058823529411764
Precision: 0.6666666666666666, Recall: 0.6666666666666666, Accuracy: 0.6842105263157895, F1 Score: 0.6666666666666666
0.6666666666666666 0.6666666666666666 0.6842105263157895 0.6666666666666666
Precision: 0.75, Recall: 0.6, Accuracy: 0.6842105263157895, F1 Score: 0.6666666666666666
0.75 0.6 0.6842105263157895 0.6666666666666666
Precision: 0.7, Recall: 0.7777777777777778, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
0.7 0.7777777777777778 0.7368421052631579 0.7368421052631579
Precision: 0.45454545454545453, Recall: 0.5555555555555556, Accuracy: 0.47368421052631576, F1 Score: 0.5
0.45454545454545453 0.5555555555555556 0.47368421052631576 0.5
Precision: 0.5714285714285714, Recall: 0.4444444444444444, Accuracy: 0.5789473684210527, F1 Score: 0.5
0.5714285714285714 0.4444444444444444 0.5789473684210527 0.5
Precisio

Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
0.7777777777777778 0.7 0.7368421052631579 0.7368421052631579
Precision: 0.625, Recall: 0.5, Accuracy: 0.5789473684210527, F1 Score: 0.5555555555555556
0.625 0.5 0.5789473684210527 0.5555555555555556
Precision: 0.8, Recall: 0.4, Accuracy: 0.631578947368421, F1 Score: 0.5333333333333333
0.8 0.4 0.631578947368421 0.5333333333333333
Precision: 0.7777777777777778, Recall: 0.7777777777777778, Accuracy: 0.7894736842105263, F1 Score: 0.7777777777777778
0.7777777777777778 0.7777777777777778 0.7894736842105263 0.7777777777777778
Precision: 0.8571428571428571, Recall: 0.6, Accuracy: 0.7368421052631579, F1 Score: 0.7058823529411765
0.8571428571428571 0.6 0.7368421052631579 0.7058823529411765
Precision: 0.5, Recall: 0.5555555555555556, Accuracy: 0.5263157894736842, F1 Score: 0.5263157894736842
0.5 0.5555555555555556 0.5263157894736842 0.5263157894736842
Precision: 0.5714285714285714, Recall: 0.4, 

Precision: 0.6666666666666666, Recall: 0.8, Accuracy: 0.6842105263157895, F1 Score: 0.7272727272727273
0.6666666666666666 0.8 0.6842105263157895 0.7272727272727273
Precision: 0.46153846153846156, Recall: 0.6666666666666666, Accuracy: 0.47368421052631576, F1 Score: 0.5454545454545454
0.46153846153846156 0.6666666666666666 0.47368421052631576 0.5454545454545454
Precision: 0.6153846153846154, Recall: 0.8888888888888888, Accuracy: 0.6842105263157895, F1 Score: 0.7272727272727273
0.6153846153846154 0.8888888888888888 0.6842105263157895 0.7272727272727273
Precision: 0.6153846153846154, Recall: 0.8, Accuracy: 0.631578947368421, F1 Score: 0.6956521739130435
0.6153846153846154 0.8 0.631578947368421 0.6956521739130435
Precision: 0.7, Recall: 0.7777777777777778, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
0.7 0.7777777777777778 0.7368421052631579 0.7368421052631579
Precision: 0.6428571428571429, Recall: 0.9, Accuracy: 0.6842105263157895, F1 Score: 0.75
0.6428571428571429 0.9 0.6842

Precision: 0.5714285714285714, Recall: 0.4, Accuracy: 0.5263157894736842, F1 Score: 0.47058823529411764
0.5714285714285714 0.4 0.5263157894736842 0.47058823529411764
Precision: 0.875, Recall: 0.7, Accuracy: 0.7894736842105263, F1 Score: 0.7777777777777778
0.875 0.7 0.7894736842105263 0.7777777777777778
Precision: 0.75, Recall: 0.6, Accuracy: 0.6842105263157895, F1 Score: 0.6666666666666666
0.75 0.6 0.6842105263157895 0.6666666666666666
Precision: 0.5833333333333334, Recall: 0.7777777777777778, Accuracy: 0.631578947368421, F1 Score: 0.6666666666666666
0.5833333333333334 0.7777777777777778 0.631578947368421 0.6666666666666666
Precision: 0.5, Recall: 0.6666666666666666, Accuracy: 0.5263157894736842, F1 Score: 0.5714285714285714
0.5 0.6666666666666666 0.5263157894736842 0.5714285714285714
Precision: 0.5555555555555556, Recall: 0.5, Accuracy: 0.5263157894736842, F1 Score: 0.5263157894736842
0.5555555555555556 0.5 0.5263157894736842 0.5263157894736842
Precision: 0.5, Recall: 0.55555555555555

Precision: 0.75, Recall: 0.6, Accuracy: 0.6842105263157895, F1 Score: 0.6666666666666666
0.75 0.6 0.6842105263157895 0.6666666666666666
Precision: 0.5, Recall: 0.7777777777777778, Accuracy: 0.5263157894736842, F1 Score: 0.6086956521739131
0.5 0.7777777777777778 0.5263157894736842 0.6086956521739131
Precision: 0.8, Recall: 0.4, Accuracy: 0.631578947368421, F1 Score: 0.5333333333333333
0.8 0.4 0.631578947368421 0.5333333333333333
Precision: 0.8, Recall: 0.8, Accuracy: 0.7894736842105263, F1 Score: 0.8
0.8 0.8 0.7894736842105263 0.8
Precision: 1.0, Recall: 0.6666666666666666, Accuracy: 0.8421052631578947, F1 Score: 0.8
1.0 0.6666666666666666 0.8421052631578947 0.8
Precision: 0.4, Recall: 0.2222222222222222, Accuracy: 0.47368421052631576, F1 Score: 0.2857142857142857
0.4 0.2222222222222222 0.47368421052631576 0.2857142857142857
Precision: 0.7777777777777778, Recall: 0.7, Accuracy: 0.7368421052631579, F1 Score: 0.7368421052631579
0.7777777777777778 0.7 0.7368421052631579 0.7368421052631579
